# RAG Scraper Prototype

**Author**  
- Tilak Parajuli

**All required links, videos, and other materials mentioned are in this drive link:**  
👉 [Google Drive Folder](https://drive.google.com/drive/folders/1yu0zuPXnAHFB1I9dqubjR2q3rSZlSf0P?usp=sharing)

# RAG-Based Company Knowledge Assistant

This notebook demonstrates a prototype of a Retrieval-Augmented Generation (RAG) system using web content from the Fusemachines website. We will:
- Scrape job listings using **Selenium + XPath**
- Preprocess the scraped data into chunks
- Embed the data using a transformer model via `ollama`
- Retrieve relevant chunks with vector search (cosine similarity)
- Generate responses using an LLM (via `ollama`)

**Tools**:
- Selenium (for scraping)
- Ollama (for embeddings and generation)
- Cosine similarity (for retrieval)

Inspired by the Hugging Face article [Code a simple RAG from scratch](https://huggingface.co/blog/ngxson/make-your-own-rag), this notebook is adapted for real-world use cases involving dynamic, web-based knowledge sources.

## Prerequisites
- Install `ollama` from [ollama.com](https://ollama.com).
- Pull the required models:
  ```bash
  ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf
  ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF

## Scrape Text
- We start by scraping job listings from the Fusemachines careers page using Selenium with Firefox.

In [8]:
# !pip install selenium

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_fusemachines_jobs():
    """
    Scrapes job listings from Fusemachines careers page using Firefox.
    
    XPaths:
    COMPANY_BUTTON_XPATH = "//button[@id='company']"
    CAREERS_LINK_XPATH = "//a[@href='/company/careers/']"
    JOB_LISTINGS_XPATH = "//div[@id='jazzhr']//div[contains(@class, 'row py-3')]"
    JOB_TITLE_XPATH = ".//div[contains(@class, 'col-md-6')]//div[@class='bold-s']"
    JOB_LOCATION_XPATH = ".//div[contains(@class, 'col-md-4')]//div[@class='c-dark-grey']"
    """
    # Set up Firefox options
    options = Options()
    # options.add_argument("--headless")  # Uncomment for headless mode
    
    # Initialize Firefox driver
    driver = webdriver.Firefox(options=options)
    
    try:
        # Maximize browser window
        driver.maximize_window()
        
        # Navigate to base URL
        driver.get("https://fusemachines.com/")
        
        # Wait for and click COMPANY dropdown
        company_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@id='company']"))
        )
        company_button.click()
        
        # Wait for and click Careers link
        careers_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='/company/careers/']"))
        )
        careers_link.click()
        
        # Wait for job listings to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "jazzhr"))
        )
        
        # Get all job listings
        job_rows = driver.find_elements(By.XPATH, "//div[@id='jazzhr']//div[contains(@class, 'row py-3')]")
        jobs = []
        
        for row in job_rows:
            # Get job title
            title = row.find_element(By.XPATH, ".//div[contains(@class, 'col-md-6')]//div[@class='bold-s']").text
            # Get location
            location = row.find_element(By.XPATH, ".//div[contains(@class, 'col-md-4')]//div[@class='c-dark-grey']").text
            jobs.append({"title": title, "location": location})
        
        # Print results
        print("Careers at Fusemachines")
        print("Available Jobs:")
        for job in jobs:
            print(f"- {job['title']} ({job['location']})")
        
        return jobs
        
    finally:
        driver.quit()

In [11]:
# if __name__ == "__main__":
#     scrape_fusemachines_jobs()

## Data Preprocessing

- We preprocess the scraped job listings to create text chunks suitable for embedding. Each job listing (title and location) is combined into a single chunk for simplicity.

In [12]:
def preprocess_jobs(jobs):
    """
    Convert job listings into text chunks for embedding.
    Each chunk is a string combining job title and location.
    """
    chunks = [f"{job['title']} in {job['location']}" for job in jobs]
    return chunks

# Example usage (run after scraping)
jobs = scrape_fusemachines_jobs()
chunks = preprocess_jobs(jobs)
print(f"Created {len(chunks)} chunks for embedding:")
for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i}: {chunk}")

Careers at Fusemachines
Available Jobs:
- Senior Product Manager (Buenos Aires, Argentina)
- UI/UX Designer (Buenos Aires, Argentina)
- Data Scientist (Argentina)
- Sr. Data Analyst (Buenos Aires, Argentina)
- Senior Fullstack Engineer (Lead) (Argentina)
- Senior Product Manager (Brasilia, Brazil)
- UI/UX Designer (Brasília, Brazil)
- Sr. Machine Learning Engineer (Brazil)
- Data Scientist (Brazil)
- Sr. Data Analyst (Brasilia, Brazil)
- Senior Fullstack Engineer (Lead) (Brazil)
- Senior Technical Project Manager (Brazil)
- Sr. Machine Learning Engineer (Toronto, Canada)
- Senior Product Manager (Chile)
- UI/UX Designer (Bogota, Colombia)
- Talent Acquisition Partner (Colombia)
- Sr. Machine Learning Engineer (Colombia)
- Data Scientist (Colombia)
- Sr. Data Analyst (Bogota, Colombia)
- Senior Fullstack Engineer (Lead) (Colombia)
- Sr. Software Engineer (Pune, India)
- Sr. Data Engineer Azure Databricks (Pune, India)
- QA Testing Engineer (Pune, India)
- Power Platform Engineer (Pune, 

## Embedding Text Chunks

- We use the bge-base-en-v1.5-gguf model via ollama to generate embeddings for each chunk and store them in an in-memory vector database.

In [13]:
# !pip install ollama
!ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 74aebb552ea7... 100% ▕████████████████▏  68 MB                         
pulling b1899e715228... 100% ▕████████████████▏  399 B                         
verifying sha256 digest 
writing manifest 
success 


In [14]:
!ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 6f85a640a97c... 100% ▕████████████████▏ 807 MB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 6c0b08d96525... 100% ▕████████████████▏   65 B                         
pulling 4549919ff315... 100% ▕████████████████▏  551 B                         
verifying sha256 digest 
writing manifest 
success 


In [15]:
import ollama

# Model names
EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

# In-memory vector database: list of (chunk, embedding) tuples
VECTOR_DB = []

def add_chunk_to_database(chunk):
    """Add a chunk and its embedding to the vector database."""
    try:
        embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
        VECTOR_DB.append((chunk, embedding))
        print(f"Embedded chunk: {chunk}")
    except Exception as e:
        print(f"Error embedding chunk '{chunk}': {e}")

# Index all chunks
for i, chunk in enumerate(chunks, 1):
    add_chunk_to_database(chunk)
    print(f"Processed {i}/{len(chunks)} chunks")

Embedded chunk: Senior Product Manager in Buenos Aires, Argentina
Processed 1/63 chunks
Embedded chunk: UI/UX Designer in Buenos Aires, Argentina
Processed 2/63 chunks
Embedded chunk: Data Scientist in Argentina
Processed 3/63 chunks
Embedded chunk: Sr. Data Analyst in Buenos Aires, Argentina
Processed 4/63 chunks
Embedded chunk: Senior Fullstack Engineer (Lead) in Argentina
Processed 5/63 chunks
Embedded chunk: Senior Product Manager in Brasilia, Brazil
Processed 6/63 chunks
Embedded chunk: UI/UX Designer in Brasília, Brazil
Processed 7/63 chunks
Embedded chunk: Sr. Machine Learning Engineer in Brazil
Processed 8/63 chunks
Embedded chunk: Data Scientist in Brazil
Processed 9/63 chunks
Embedded chunk: Sr. Data Analyst in Brasilia, Brazil
Processed 10/63 chunks
Embedded chunk: Senior Fullstack Engineer (Lead) in Brazil
Processed 11/63 chunks
Embedded chunk: Senior Technical Project Manager in Brazil
Processed 12/63 chunks
Embedded chunk: Sr. Machine Learning Engineer in Toronto, Canada


## Implementing Similarity Search
- We implement a retrieval function using cosine similarity to find the top N most relevant chunks for a given query.

In [16]:
def cosine_similarity(a, b):
    """Calculate cosine similarity between two vectors."""
    dot_product = sum(x * y for x, y in zip(a, b))
    norm_a = sum(x ** 2 for x in a) ** 0.5
    norm_b = sum(x ** 2 for x in b) ** 0.5
    return dot_product / (norm_a * norm_b) if norm_a and norm_b else 0

def retrieve(query, top_n=3):
    """Retrieve top N most relevant chunks based on query."""
    try:
        query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
        similarities = []
        for chunk, embedding in VECTOR_DB:
            similarity = cosine_similarity(query_embedding, embedding)
            similarities.append((chunk, similarity))
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_n]
    except Exception as e:
        print(f"Error retrieving query '{query}': {e}")
        return []

# Example retrieval
query = "Are there any Data Scientist or AI related jobs open at fusemachines canada?"
retrieved = retrieve(query)
print("\nExample retrieval for query:", query)
for chunk, similarity in retrieved:
    print(f" - (similarity: {similarity:.2f}) {chunk}")


Example retrieval for query: Are there any Data Scientist or AI related jobs open at fusemachines canada?
 - (similarity: 0.78) Sr. Machine Learning Engineer in Toronto, Canada
 - (similarity: 0.68) Data Scientist in Argentina
 - (similarity: 0.67) Data Scientist in Kathmandu, Nepal


## Inference: Response Generation with Language Model
- We use the Llama-3.2-1B-Instruct-GGUF model to generate responses based on the retrieved chunks.

In [17]:
def generate_response(query, retrieved_knowledge):
    """Generate a response using the language model and retrieved knowledge."""
    instruction_prompt = f"""You are a helpful chatbot providing information about job openings at Fusemachines.
Use only the following job listings to answer the question. Don't make up any new information:
{'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}
"""
    try:
        stream = ollama.chat(
            model=LANGUAGE_MODEL,
            messages=[
                {'role': 'system', 'content': instruction_prompt},
                {'role': 'user', 'content': query},
            ],
            stream=True,
        )
        print('Chatbot response:')
        response = ''
        for chunk in stream:
            content = chunk['message']['content']
            print(content, end='', flush=True)
            response += content
        print()  # New line after response
        return response
    except Exception as e:
        print(f"Error generating response for query '{query}': {e}")
        return ""

# Example generation
response = generate_response(query, retrieved)

Chatbot response:
Unfortunately, I don't have real-time access to current job openings on Fusemachines. However, I can suggest some options to help you find the information you're looking for:

1. **Visit the Fusemachines website**: You can check their official job board or career section to see if they have any data scientist or AI-related jobs available.
2. **Job search platforms**: Websites like Indeed, LinkedIn, and Glassdoor may also have Fusemachines' job listings. Try searching for "Fusemachines" along with the desired job title (e.g., Data Scientist).
3. **Contact Fusemachines directly**: You can reach out to Fusemachines' HR department or a specific hiring manager to inquire about available data scientist or AI-related positions.

Regarding Sr. Machine Learning Engineer, I couldn't find any information on this role at Fusemachines. If you're interested in applying for the position, it's likely that they have other openings for senior engineers with different expertise.

For Da

## Testing and Validation
- We test the RAG system with a set of queries to validate its performance and ensure it retrieves relevant jobs and generates accurate responses.

In [18]:
def test_rag_system():
    """Test the RAG system with sample queries."""
    test_queries = [
        "What data scientist jobs are available?",
        "Are there any jobs in Canada?",
        "Tell me about software engineer positions",
        "What roles are available in Nepal?",
    ]
    
    for query in test_queries:
        print(f"\nTesting query: {query}")
        print("Retrieved job listings:")
        retrieved = retrieve(query)
        for chunk, similarity in retrieved:
            print(f" - (similarity: {similarity:.2f}) {chunk}")
        generate_response(query, retrieved)
        print("-" * 50)

# Run tests
test_rag_system()

# Interactive loop for further testing
print("\nInteractive Testing")
while True:
    query = input("Ask about Fusemachines job openings (or type 'exit' to quit): ")
    if query.lower() == 'exit':
        break
    print("\nRetrieved job listings:")
    retrieved = retrieve(query)
    for chunk, similarity in retrieved:
        print(f" - (similarity: {similarity:.2f}) {chunk}")
    generate_response(query, retrieved)



Testing query: What data scientist jobs are available?
Retrieved job listings:
 - (similarity: 0.77) Data Scientist in Kathmandu, Nepal
 - (similarity: 0.74) Data Scientist in Argentina
 - (similarity: 0.73) Data Scientist in Brazil
Chatbot response:
Unfortunately, I don't have any data on specific jobs that match your criteria (Kathmandu, Nepal; Argentina; or Brazil). However, I can provide you with general information about data science job openings at Fusemachines.

Fusemachines is a company that specializes in machine learning and artificial intelligence solutions. They often hire data scientists to work on various projects, including data engineering, modeling, and deployment.

As of my knowledge cutoff, here are some job categories that might be relevant:

* Data Scientist: We occasionally have openings for skilled data scientists who can build and deploy models using popular machine learning libraries.
* Data Engineer: Fusemachines also hires data engineers to help design, deve

KeyboardInterrupt: Interrupted by user

## Conclusion
This notebook implements a RAG system that:

Scrapes job listings from Fusemachines using Selenium.
- Preprocesses the data into chunks.
- Embeds chunks using ollama’s bge-base-en-v1.5-gguf model.
- Retrieves relevant jobs using cosine similarity.
- Generates responses with Llama-3.2-1B-Instruct-GGUF.

## Improvements:

- Use a scalable vector database (e.g., Qdrant, Pinecone).
- Implement reranking for better retrieval.
- Scrape additional job details (e.g., descriptions) for richer context.
- Use a larger LLM for improved response quality.